In [1]:
import spacy
import os
import torch
from transformers import AutoTokenizer
from tqdm import tqdm
import torch

In [2]:
from scripts.utils import load_model
model_name = 'falcon7b'
tokenizer, _ = load_model(f'{model_name}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
import pandas as pd
metric_data_path = '/scratch/ramprasad.sa/probing_summarization_factuality/metric_scores/scores_falcon7b_XSUM.csv'
df = pd.read_csv(metric_data_path)
df.head()

,Unnamed: 0,target_token_id,factual_label,target_token_str,P_y|x,P_y,P_ytopic,score_pmi,score_pmi_topic,distance_pmi,...,score_poel,score_poel_topic,score_harim,score_harim_topic,score_harim_plus,score_harim_plus_topic,score_coco_token,score_coco_span,score_coco_sent,document_id
0,0,1440,0,On,"[1.058425667110896e-07, 1.1950911614488913e-11...","[2.834036649801419e-07, 2.9363022235884273e-07...","[1.2684520100947339e-08, 4.711731352302628e-11...",-0.680602,0.735569,1.977682,...,0.902703,1.831712,0.904734,0.934342,-9.255168,-9.462425,0.975828,0.958098,0.956422,Cynthia Lamanda#XSUM-34443723#1701418173:falco...
1,1,204,0,,"[8.021726927154305e-08, 9.161906650012597e-11,...","[2.8300743692977193e-11, 3.2990399301269235e-1...","[4.6786907681450884e-09, 1.431930407580495e-10...",-1.518157,0.454188,1.886547,...,-0.507458,0.786400,0.135744,0.248125,-1.370127,-2.156797,1.000000,1.000000,1.000000,Cynthia Lamanda#XSUM-34443723#1701418173:falco...
2,2,1392,0,17,"[7.483722441747886e-08, 2.9117004363143906e-09...","[2.2218439710286475e-07, 2.2523076737002157e-0...","[2.4044721769200805e-08, 7.306030003206843e-09...",-2.350952,0.947837,1.928241,...,-1.197902,0.966103,0.000034,0.000889,-0.001173,-0.007160,0.051838,0.046032,0.044262,Cynthia Lamanda#XSUM-34443723#1701418173:falco...
3,3,3175,0,June,"[3.743747534201702e-09, 1.020715290067642e-09,...","[1.1662526500089143e-09, 4.248814355900521e-10...","[2.7728994100861826e-10, 2.1049970100328608e-1...",-1.937539,-2.331730,1.901794,...,-0.927529,-1.186119,-0.000076,-0.000050,0.001994,0.001816,0.050100,0.066953,0.055005,Cynthia Lamanda#XSUM-34443723#1701418173:falco...
4,4,23,0,",","[2.063961224152422e-09, 2.547753674697617e-12,...","[2.6789446216923807e-09, 1.1517421044660026e-1...","[1.436949892408279e-09, 2.520010589091015e-10,...",0.608423,-0.336032,0.189328,...,1.210392,0.590829,0.681027,0.602127,-6.125522,-5.573220,1.000000,1.000000,1.000000,Cynthia Lamanda#XSUM-34443723#1701418173:falco...


In [16]:
len(df[df['target_token_id'] == 204])

77

In [10]:
text = ''.join(list(df['target_token_str'].values)[:50])

In [11]:
tokenizer(text)

{'input_ids': [1440, 204, 1392, 3175, 23, 241, 12593, 9346, 38785, 2729, 5131, 423, 23, 1321, 398, 15081, 427, 34276, 18, 94, 20318, 8938, 25, 5878, 1658, 646, 614, 12270, 275, 248, 17191, 23, 273, 241, 2914, 923, 451, 2301, 272, 3520, 313, 248, 1144, 1002, 25, 22818, 304, 11558, 263, 241], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
list(df['target_token_id'].values)[:50]

[1440,
 204,
 1392,
 3175,
 23,
 241,
 12593,
 9346,
 38785,
 2729,
 5131,
 423,
 23,
 1321,
 398,
 15081,
 427,
 34276,
 18,
 94,
 20318,
 8938,
 25,
 5878,
 1658,
 646,
 614,
 12270,
 275,
 248,
 17191,
 23,
 273,
 241,
 2914,
 923,
 451,
 2301,
 272,
 3520,
 313,
 248,
 1144,
 1002,
 25,
 22818,
 304,
 11558,
 263,
 241]

In [63]:
text = 'Complete the following text:\nDirector Sanjay Leela'
inputs = tokenizer(text, return_tensors="pt").to('cuda')
outputs = model.generate(**inputs,
                max_length = inputs['input_ids'].shape[-1] + 1,
               output_attentions = True,
                        output_hidden_states=True, 
                        return_dict_in_generate=True)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [64]:
source_len = inputs['input_ids'].shape[-1]

In [65]:
hidden_states = torch.cat(outputs['hidden_states'][-1])
hidden_states.shape
summary_hidden_states = hidden_states[:, source_len - 1, :]

In [66]:
summary_hidden_states.shape

torch.Size([33, 4096])

In [67]:
import torch.nn.functional as F
logits = model.lm_head(summary_hidden_states)
probs = F.softmax(logits, dim=-1).cpu().detach()

In [74]:
torch.tensor([torch.argmax(probs[25]).item()]), torch.max(probs[25])

(tensor([24442]), tensor(5.5836e-05))

In [69]:
outputs

GenerateDecoderOnlyOutput(sequences=tensor([[    1, 21929,   272,  2296,  2245, 28747,    13,  7746,   271,  3652,
         28768,   339,  1337,  4403, 18811]], device='cuda:0'), scores=None, logits=None, attentions=((tensor([[[[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9388e-01, 6.1180e-03, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [8.5309e-01, 1.1106e-02, 1.3580e-01,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [8.3430e-01, 2.1248e-03, 9.4038e-02,  ..., 1.0516e-02,
           0.0000e+00, 0.0000e+00],
          [9.2368e-01, 8.9840e-04, 5.1755e-02,  ..., 2.2319e-03,
           3.5004e-03, 0.0000e+00],
          [7.5690e-01, 4.1897e-03, 1.2864e-01,  ..., 2.4518e-03,
           4.3347e-03, 1.3725e-02]],

         [[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.9810e-01, 1.8968e-03, 0.0000e+00,  ..., 0.000

In [2]:
model_path = {'mistral7b': 'mistralai/Mistral-7B-Instruct-v0.1',
             'falcon7b': 'tiiuae/falcon-7b-instruct',
              'llama7b': '/work/frink/models/Llama-2-7b-chat-hf',
             'flanul2': 'google/flan-ul2'}

def load_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_path[model_name],
                                         cache_dir = '/scratch/ramprasad.sa/huggingface_models')
    return tokenizer

In [3]:
model_name = 'falcon7b'
folder_path = f'/scratch/ramprasad.sa/probing_summarization_factuality/internal_states/USB/XSUM/{model_name}/document_context'
tokenizer = load_tokenizer('falcon7b')

In [4]:
files = os.listdir(folder_path)
files = sorted(files)
    
for file_name in tqdm(files[:10]):
    docid = file_name.split('#')[-1].split('.pt')[0]
    if os.path.isfile(os.path.join(folder_path, file_name)):
        example = torch.load(os.path.join(folder_path, file_name))
        source_len = example['source_len']
        all_tokens = example['all_tokens']
        print(tokenizer.decode(all_tokens[source_len:]))
        print('***' * 13)
        break

  0%|                                                                                                                                                                                                                  | 0/10 [00:01<?, ?it/s]

Director Sanjay Leela Bhansali wants to make a film on Indian Olympian boxer MC Mary Kom, who hails from the northeast state of Manipur.. The boxer says the project will help bridge the gap between her region and the rest of India, while Bhansali described her as an "inspiration to every Indian woman".
***************************************


In [7]:
tokenizer.decode(all_tokens[1])

' a'

In [17]:
example['all_tokens']

tensor([50797,   241, 11055,   312,   248,  1863,  3268,   272,  5849,    25,
         1699,  3957,   248, 11055,    23,   736,   745,  1150,   325,   304,
         1652,   272,   248,  3268,   193, 16199,    37, 27729, 20591, 23437,
        64148,   383,    23, 48013, 14938,  5530, 16169,    23,   504, 17623,
          241,  2675,   325, 37697,  5219,  3787, 42402,  1135,  4497, 13384,
          501,  5748,  4045,   271,   717,   313,   573,  1063,   513,   204,
           13,  1182,  2675,   451,   733,  9336,   248,  9939,  1192,   692,
          275,   248,  4673, 32921,   273,  1014,   427,   248,  6775,    24,
           80,  8546,  3800,  1311,  5530, 16169,  2276,   248, 15097,   513,
          390,  2505,    24,  2270,  1079, 12969,   304,   427,   248,  6969,
        35584,  1592,   275,  1676, 25691,   513, 13384,   501,  5748,    23,
          569,   504,   650,  4042,   248,  2675,   312,   241,   649,    23,
         5383,   248, 48013,   345,   267,   204,    13,   965, 

In [8]:
tokens

['The',
 'case',
 'against',
 'two',
 'men',
 'accused',
 'of',
 'stealing',
 '£',
 '20,000',
 'of',
 'biscuits',
 'from',
 'the',
 'makers',
 'of',
 'Jammie',
 'Dodgers',
 'and',
 'Wagon',
 'Wheels',
 'has',
 'been',
 'dropped',
 '..',
 'A',
 'trailer',
 'was',
 'taken',
 'from',
 'Burton',
 "'s",
 'Foods',
 'Ltd',
 'on',
 'Ty',
 'Coch',
 'Way',
 ',',
 'Cwmbran',
 ',',
 'Torfaen',
 'on',
 '17',
 'June',
 '..',
 'The',
 'two',
 'men',
 ',',
 'from',
 'Liverpool',
 'and',
 'Cheshire',
 ',',
 'had',
 'been',
 'charged',
 'with',
 'theft',
 '..',
 'A',
 'hearing',
 'at',
 'Newport',
 'Magistrates',
 "'",
 'Court',
 'heard',
 'a',
 'third',
 'man',
 'will',
 'appear',
 'in',
 'court',
 'in',
 'relation',
 'to',
 'the',
 'alleged',
 'theft',
 'on',
 'Friday',
 '.']